In [3]:
from multiprocessing import Pool
from functools import partial
import os
import numpy as np

from mdp_sequence_reader import MDPSequenceReader
from training_pomc_util import *
from pomc import PartiallyObservableMarkovChain

In [227]:
# configuration
sequence_root_dir = 'sequence'
pomdp_setup_root_dir = 'pomdp_setup'
scenario_name = 'sharp_0.20_0.08'

max_em_iterations = 50

min_num_states = 2
max_num_states = 17
model_retries = 6

In [228]:
# read the sequence
reader = MDPSequenceReader(os.path.join(sequence_root_dir, scenario_name + '.txt'))
print("Num states: %d" % reader.get_num_states())
print("Num steps: %d" % reader.get_num_steps())

Num states: 12
Num steps: 10000


In [229]:
# generate environment file
pomdp_setup_folder = os.path.join(pomdp_setup_root_dir, scenario_name)
if not os.path.exists(pomdp_setup_folder):
    os.makedirs(pomdp_setup_folder)
reader.generate_grid_world_setup_file(os.path.join(pomdp_setup_folder, 'world.txt'))

In [230]:
# get the sequences
observations = reader.get_observation_sequence()
actions = reader.get_action_sequence()

In [231]:
# get the initial model
action_space = [0, 1, 2, 3]   # (up, right, down, left)
num_observables = 16   # binary measurements of 4 directions

In [232]:
# model training
def train_model(num_states, round_idx, action_space, num_observables, actions, observations,
                converge_improvement_threshold=2., converge_improve_retry=3):
    
    # initialize model
    init_model = initialize_random_pomc(num_states, action_space, num_observables)
    
    # iterate the model
    model = init_model
    log_likelihood = -1e100  # very small
    best_log_likelihood = log_likelihood
    convergence_count = 0
    for r in range(max_em_iterations):
        alist, c = improve_params(xs=actions, ys=observations, m=model)
        new_model = PartiallyObservableMarkovChain(alist, c, model.init)

        next_log_likelihood = get_log_likelihood(make_tableaus(xs=actions, ys=observations, m=new_model))
        #print("round=%d, log likelihood=%f" % (r+1, next_log_likelihood))

        model = new_model
        log_likelihood = next_log_likelihood
    
        # check convergence condition
        if log_likelihood - best_log_likelihood > converge_improvement_threshold:
            convergence_count = 0
        else:
            convergence_count += 1
            if convergence_count == converge_improve_retry:
                break
        
        best_log_likelihood = max(best_log_likelihood, log_likelihood)
    
    print("num_states=%d, round_idx=%d, log_likelihood=%f" % (num_states, round_idx, log_likelihood))
    return num_states, round_idx,  model, log_likelihood, best_log_likelihood

In [233]:
# generate task info for multi-processing
tasks = []
for i_num_states in range(min_num_states, max_num_states+1):
    for i_round in range(model_retries):
        tasks.append((i_num_states, i_round))

In [ ]:
# do it so fast!
partially_fed_train_model_func = partial(
    train_model,
    action_space=action_space,
    num_observables=num_observables,
    actions=actions,
    observations=observations,
)
results = Pool().starmap(partially_fed_train_model_func, tasks)

num_states=2, round_idx=0, log_likelihood=-24339.907531
num_states=2, round_idx=3, log_likelihood=-24339.907531
num_states=3, round_idx=0, log_likelihood=-22978.695879
num_states=3, round_idx=3, log_likelihood=-22978.695879
num_states=5, round_idx=0, log_likelihood=-20409.602664
num_states=5, round_idx=3, log_likelihood=-20409.602664
num_states=4, round_idx=3, log_likelihood=-21773.721911
num_states=4, round_idx=0, log_likelihood=-21773.721911
num_states=2, round_idx=1, log_likelihood=-24220.189968
num_states=2, round_idx=4, log_likelihood=-24220.189968
num_states=3, round_idx=1, log_likelihood=-22651.847318
num_states=3, round_idx=4, log_likelihood=-22651.847318
num_states=5, round_idx=1, log_likelihood=-20423.821753
num_states=5, round_idx=4, log_likelihood=-20423.821753
num_states=4, round_idx=4, log_likelihood=-21642.865385
num_states=4, round_idx=1, log_likelihood=-21642.865385
num_states=3, round_idx=2, log_likelihood=-22963.322091
num_states=3, round_idx=5, log_likelihood=-22963

In [ ]:
result_table = [[None for _ in range(model_retries)] for _ in range(max_num_states+1)]
for result in results:
    num_states, round_idx, *rest = result
    result_table[num_states][round_idx] = rest
    
pomc_models_folder = os.path.join(pomdp_setup_folder, 'pomc_models')
if not os.path.exists(pomc_models_folder):
    os.makedirs(pomc_models_folder)
for i_num_states in range(min_num_states, max_num_states+1):
    best_model_idx = np.argmax([logl for _, logl, _ in result_table[i_num_states]])
    best_model, best_final_likelihood, _ = result_table[i_num_states][best_model_idx]
    print("%d states, best likelihood=%f" % (i_num_states, best_final_likelihood))
    best_model.dump(os.path.join(pomc_models_folder, "best_%d_states.txt" % i_num_states))
for i_num_states in range(min_num_states, max_num_states+1):
    best_model_idx = np.argmax([logl for _, logl, _ in result_table[i_num_states]])
    best_model, best_final_likelihood, _ = result_table[i_num_states][best_model_idx]
    print("    %f,  # %d states" % (best_final_likelihood, i_num_states))